# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 
The CNN architecture is a transfer learning network based on RestNet50 model.  I don't make any change to the model provided with the project.

The RNN Decoder architecture is as follow:
I used an embedding layer to map the captions to embedded word vector. I also used an LSTM layer with output of the CNN encoder and embedded word vector as inputs. Finally I use a Fully Connected Layer to map the hidden layer from the lstm outputs to the predicted word. 
I decided to init the weights with xavier normal initializer to make the LSTM model performing better.
Firstly, I delete the last column of the captions vectors. Then I fed it into the embedding layer. I then concatenate the features vector with the captions output from the embedding layer. I then pass it through the lstm layer and the FC Layer. I chose 512 for the embed size and hidden size as mentionned in the paper **Show and Tell A neural image caption generator**. https://arxiv.org/pdf/1411.4555.pdf


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I didn't change the transform provided with the project. This tranform scales the images to a correct size, normalizes the images and applies standard pre-processing transforms like random horizontal flipping, converts the processed image to a tensor and finally normalize it as in the pytorch documentation, so that it can be provided as CNN input.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** The trainable parameters selected for training are the weights in the embedding layer of the encoder and all trainable weights in the decoder. The pretrained weights in the encoder are not altered so as to preserve the tuning from its pretraining. It is interesting to recall that using pretrained weights have been reported to reduce overfitting.

### Task #

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** I decide to use Adam Optimizer. Adam is a good choice because it combines the benefits of Adagrad and RMSProp and is better than SGD on this problem.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 4        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# init the weights of the rnn
decoder.init_weights()

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 

# TODO #4: Define the optimizer.
optimizer = optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...




  0%|          | 0/414113 [00:00<?, ?it/s]

  0%|          | 345/414113 [00:00<02:00, 3448.04it/s]

  0%|          | 777/414113 [00:00<01:52, 3670.22it/s]

Done (t=0.87s)
creating index...
index created!
Obtaining caption lengths...




  0%|          | 1216/414113 [00:00<01:47, 3858.47it/s]

  0%|          | 1663/414113 [00:00<01:42, 4022.68it/s]

  1%|          | 2105/414113 [00:00<01:39, 4133.67it/s]

  1%|          | 2541/414113 [00:00<01:38, 4196.61it/s]

  1%|          | 2969/414113 [00:00<01:37, 4219.51it/s]

  1%|          | 3404/414113 [00:00<01:36, 4256.77it/s]

  1%|          | 3853/414113 [00:00<01:34, 4323.07it/s]

  1%|          | 4298/414113 [00:01<01:33, 4360.04it/s]

  1%|          | 4748/414113 [00:01<01:33, 4401.06it/s]

  1%|▏         | 5199/414113 [00:01<01:32, 4431.00it/s]

  1%|▏         | 5638/414113 [00:01<01:33, 4391.66it/s]

  1%|▏         | 6092/414113 [00:01<01:31, 4435.11it/s]

  2%|▏         | 6534/414113 [00:01<01:32, 4391.73it/s]

  2%|▏         | 6984/414113 [00:01<01:32, 4421.33it/s]

  2%|▏         | 7426/414113 [00:01<01:32, 4401.39it/s]

  2%|▏         | 7878/414113 [00:01<01:31, 4429.03it/s]

  2%|▏         | 8327/414113 [00:01<01:31, 4447.00it/s]

  2%|▏         | 8772/414113 

 28%|██▊       | 115973/414113 [00:26<01:05, 4543.47it/s]

 28%|██▊       | 116431/414113 [00:26<01:05, 4553.93it/s]

 28%|██▊       | 116887/414113 [00:26<01:05, 4531.44it/s]

 28%|██▊       | 117341/414113 [00:26<01:05, 4520.85it/s]

 28%|██▊       | 117804/414113 [00:26<01:05, 4550.12it/s]

 29%|██▊       | 118260/414113 [00:26<01:05, 4509.16it/s]

 29%|██▊       | 118712/414113 [00:26<01:05, 4501.85it/s]

 29%|██▉       | 119163/414113 [00:27<01:05, 4485.81it/s]

 29%|██▉       | 119613/414113 [00:27<01:05, 4490.00it/s]

 29%|██▉       | 120070/414113 [00:27<01:05, 4511.02it/s]

 29%|██▉       | 120522/414113 [00:27<01:06, 4432.68it/s]

 29%|██▉       | 120982/414113 [00:27<01:05, 4480.38it/s]

 29%|██▉       | 121431/414113 [00:27<01:05, 4470.53it/s]

 29%|██▉       | 121890/414113 [00:27<01:04, 4504.25it/s]

 30%|██▉       | 122341/414113 [00:27<01:05, 4475.44it/s]

 30%|██▉       | 122798/414113 [00:27<01:04, 4501.48it/s]

 30%|██▉       | 123249/414113 [00:27<01:04, 4487.65it/s

 55%|█████▌    | 227971/414113 [00:52<00:40, 4590.91it/s]

 55%|█████▌    | 228436/414113 [00:52<00:40, 4605.97it/s]

 55%|█████▌    | 228897/414113 [00:52<00:40, 4580.63it/s]

 55%|█████▌    | 229356/414113 [00:52<00:40, 4538.85it/s]

 55%|█████▌    | 229811/414113 [00:52<00:41, 4459.98it/s]

 56%|█████▌    | 230258/414113 [00:52<00:41, 4423.88it/s]

 56%|█████▌    | 230701/414113 [00:52<00:41, 4370.81it/s]

 56%|█████▌    | 231142/414113 [00:52<00:41, 4381.34it/s]

 56%|█████▌    | 231581/414113 [00:52<00:41, 4383.31it/s]

 56%|█████▌    | 232020/414113 [00:53<00:41, 4363.15it/s]

 56%|█████▌    | 232462/414113 [00:53<00:41, 4377.55it/s]

 56%|█████▌    | 232903/414113 [00:53<00:41, 4386.66it/s]

 56%|█████▋    | 233352/414113 [00:53<00:40, 4416.37it/s]

 56%|█████▋    | 233810/414113 [00:53<00:40, 4462.98it/s]

 57%|█████▋    | 234257/414113 [00:53<00:40, 4457.83it/s]

 57%|█████▋    | 234708/414113 [00:53<00:40, 4472.98it/s]

 57%|█████▋    | 235163/414113 [00:53<00:39, 4493.56it/s

 83%|████████▎ | 341703/414113 [01:17<00:16, 4454.49it/s]

 83%|████████▎ | 342150/414113 [01:17<00:16, 4456.69it/s]

 83%|████████▎ | 342598/414113 [01:18<00:16, 4462.46it/s]

 83%|████████▎ | 343045/414113 [01:18<00:16, 4423.46it/s]

 83%|████████▎ | 343499/414113 [01:18<00:15, 4457.04it/s]

 83%|████████▎ | 343954/414113 [01:18<00:15, 4481.94it/s]

 83%|████████▎ | 344407/414113 [01:18<00:15, 4494.65it/s]

 83%|████████▎ | 344857/414113 [01:18<00:15, 4495.32it/s]

 83%|████████▎ | 345308/414113 [01:18<00:15, 4498.98it/s]

 83%|████████▎ | 345758/414113 [01:18<00:16, 4241.91it/s]

 84%|████████▎ | 346207/414113 [01:18<00:15, 4312.98it/s]

 84%|████████▎ | 346646/414113 [01:18<00:15, 4333.10it/s]

 84%|████████▍ | 347082/414113 [01:19<00:16, 4175.40it/s]

 84%|████████▍ | 347522/414113 [01:19<00:15, 4239.26it/s]

 84%|████████▍ | 347959/414113 [01:19<00:15, 4275.99it/s]

 84%|████████▍ | 348402/414113 [01:19<00:15, 4319.52it/s]

 84%|████████▍ | 348850/414113 [01:19<00:14, 4364.54it/s

<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [4]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/6471], Loss: 3.8046, Perplexity: 44.90948
Epoch [1/3], Step [200/6471], Loss: 3.3057, Perplexity: 27.2680
Epoch [1/3], Step [300/6471], Loss: 3.1511, Perplexity: 23.3621
Epoch [1/3], Step [400/6471], Loss: 3.1798, Perplexity: 24.0409
Epoch [1/3], Step [500/6471], Loss: 3.0287, Perplexity: 20.6710
Epoch [1/3], Step [600/6471], Loss: 2.9684, Perplexity: 19.4612
Epoch [1/3], Step [700/6471], Loss: 2.6641, Perplexity: 14.3555
Epoch [1/3], Step [800/6471], Loss: 3.0270, Perplexity: 20.6354
Epoch [1/3], Step [900/6471], Loss: 2.8970, Perplexity: 18.1202
Epoch [1/3], Step [1000/6471], Loss: 2.8893, Perplexity: 17.9803
Epoch [1/3], Step [1100/6471], Loss: 3.4268, Perplexity: 30.7791
Epoch [1/3], Step [1200/6471], Loss: 2.7623, Perplexity: 15.8366
Epoch [1/3], Step [1300/6471], Loss: 2.3226, Perplexity: 10.2020
Epoch [1/3], Step [1400/6471], Loss: 2.8598, Perplexity: 17.4580
Epoch [1/3], Step [1500/6471], Loss: 2.4659, Perplexity: 11.7746
Epoch [1/3], Step [1600/6471], Lo

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch [2/3], Step [6000/6471], Loss: 1.9646, Perplexity: 7.13209
Epoch [2/3], Step [6200/6471], Loss: 1.8788, Perplexity: 6.54544
Epoch [2/3], Step [6300/6471], Loss: 2.2083, Perplexity: 9.10024
Epoch [2/3], Step [6400/6471], Loss: 2.0062, Perplexity: 7.43484
Epoch [3/3], Step [100/6471], Loss: 2.0410, Perplexity: 7.698302
Epoch [3/3], Step [200/6471], Loss: 1.9563, Perplexity: 7.07330
Epoch [3/3], Step [300/6471], Loss: 2.0855, Perplexity: 8.04839
Epoch [3/3], Step [400/6471], Loss: 2.2469, Perplexity: 9.45883
Epoch [3/3], Step [500/6471], Loss: 1.7991, Perplexity: 6.04416
Epoch [3/3], Step [600/6471], Loss: 1.9842, Perplexity: 7.27303
Epoch [3/3], Step [700/6471], Loss: 1.9748, Perplexity: 7.20487
Epoch [3/3], Step [800/6471], Loss: 1.9426, Perplexity: 6.97711
Epoch [3/3], Step [900/6471], Loss: 1.8661, Perplexity: 6.46328
Epoch [3/3], Step [1000/6471], Loss: 1.9728, Perplexity: 7.1905
Epoch [3/3], Step [1100/6471], Loss: 2.2337, Perplexity: 9.33427
Epoch [3/3], Step [1200/6471], Los

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch [3/3], Step [4400/6471], Loss: 2.0328, Perplexity: 7.63530
Epoch [3/3], Step [4500/6471], Loss: 2.1587, Perplexity: 8.66004
Epoch [3/3], Step [4600/6471], Loss: 1.9009, Perplexity: 6.69188
Epoch [3/3], Step [4700/6471], Loss: 2.2160, Perplexity: 9.17055
Epoch [3/3], Step [4800/6471], Loss: 1.7359, Perplexity: 5.67382
Epoch [3/3], Step [4900/6471], Loss: 1.8817, Perplexity: 6.56479
Epoch [3/3], Step [5000/6471], Loss: 2.0368, Perplexity: 7.66593
Epoch [3/3], Step [5100/6471], Loss: 1.8570, Perplexity: 6.40435
Epoch [3/3], Step [5300/6471], Loss: 2.2576, Perplexity: 9.55978
Epoch [3/3], Step [5400/6471], Loss: 1.8430, Perplexity: 6.31555
Epoch [3/3], Step [5500/6471], Loss: 2.2422, Perplexity: 9.41405
Epoch [3/3], Step [5600/6471], Loss: 1.8089, Perplexity: 6.10354
Epoch [3/3], Step [5700/6471], Loss: 2.0064, Perplexity: 7.43638
Epoch [3/3], Step [5800/6471], Loss: 1.9837, Perplexity: 7.26962
Epoch [3/3], Step [5900/6471], Loss: 1.7924, Perplexity: 6.00408
Epoch [3/3], Step [6000/6

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.